In [ ]:
! pip install transformers
! pip install datasets
!pip install torchmetrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.2 MB/s 
     |████████████████████████████████| 120 kB 85.5 MB/s 
     |████████████████████████████████| 6.6 MB 51.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 8.2 MB/s 
     |████████████████████████████████| 212 kB 77.1 MB/s 
     |████████████████████████████████| 115 kB 59.1 MB/s 
     |████████████████████████████████| 127 kB 95.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 8.9 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time

from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.utils import shuffle
import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation

Mounted at /content/drive
/content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation


In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:

df_unsupervised = pd.read_json('/content/drive/MyDrive/新建文件夹/ERAI_Dataset_unsupervised.json')

In [ ]:
df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
# df2 = pd.read_json('ERAI_Dataset_pairwise_test.json')
df.reset_index(drop = True,inplace = True)
df = df.dropna(axis = 0)



df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))


In [ ]:
df

In [ ]:
PRETRAINED_MODEL_NAME = 'yiyanghkust/finbert-tone'
# 获取预测模型所使用的tokenizer

#"nlpaueb/sec-bert-shape"
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [ ]:

def generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2):
  
  
  first_all_value = pd.concat([mpp_value1,ml_value1],axis=1)
  second_all_value = pd.concat([mpp_value2,ml_value2],axis=1)
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'mpp_label':torch.tensor(mpp_label.values,dtype = torch.long),
      'ml_label':torch.tensor(ml_label.values,dtype = torch.long),
      'first_all_value': torch.tensor(first_all_value.values, dtype=torch.float),
      'second_all_value': torch.tensor(second_all_value.values, dtype=torch.float),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

    

In [ ]:
class BertNerClassifier(nn.Module):

  def __init__(self, n_classes_cls,n_classes_reg):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained('yiyanghkust/finbert-tone')
    self.drop = nn.Dropout(p=0.1)
    self.out_cls = nn.Linear(self.bert.config.hidden_size, n_classes_cls)

    self.out_reg = nn.Linear(self.bert.config.hidden_size, n_classes_reg)
    self.sigmoid = nn.Sigmoid()

  
  def forward(self, input_ids, attention_mask,token_type_ids,max_len,length1,length2):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids = token_type_ids

    )
    
    output = self.drop(output[0])
    
    cls_info = output[0][0]
    sent1_info = torch.mean(output[0][1:(length1+1)],dim=0)
    sent2_info = torch.mean(output[0][length1+2:(length1+2+length2)],dim=0)
    
    # output = torch.cat((cls_info,sent1_info,sent2_info),dim = 0)
    cls_info = self.out_cls(cls_info)
    
    
    
    return self.sigmoid(cls_info),self.out_reg(sent1_info),self.out_reg(sent2_info)

In [ ]:
 
class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']
      self.mpp_label = data['mpp_label']
      self.ml_label = data['ml_label']
      self.first_all_value = data['first_all_value']
      self.second_all_value = data['second_all_value']
      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.mpp_label)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.mpp_label[index],self.ml_label[index],self.first_all_value[index],self.second_all_value[index],self.first_sen_len[index],self.second_sen_len[index]

In [ ]:
def generate_dataloader(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2,batch_size):
  data = generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader


In [ ]:
EPOCHS = 50

# total_steps = len(train_data_loader) * EPOCHS

# scheduler = get_linear_schedule_with_warmup(
#   optimizer,
#   num_warmup_steps=0,
#   num_training_steps=total_steps
# )
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_reg = nn.MSELoss().to(device)
loss_cls = nn.BCELoss().to(device)






In [ ]:

def train_and_eval(model, 
  train_loader, 
  validation_loader, 
  optimizer,  
  device,
  epoch,
  # label_num 
  ):
  model.train()
  max_len = 256
  mpp_accuracy_state = 0
  ml_accuracy_state = 0
  for epo in range(epoch):
    
    train_loss = []
    
    global train_mpp_predictions 
    train_mpp_predictions = []
    global train_ml_predictions
    train_ml_predictions = []
    

    validation_loss = []
    
    validation_mpp_predictions = []
    validation_ml_predictions = []


    global real_train_mpp 
    real_train_mpp = []
    global real_train_ml
    real_train_ml = []
    real_validation_mpp = []
    real_validation_ml = []
    
   
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      token_type_ids = data[2].to(device)
      
      mpp_targets = data[3].to(device)
      ml_targets = data[4].to(device)
      first_targets = data[5].to(device)
      second_targets = data[6].to(device)
      first_sen_len = data[7].to(device)
      second_sen_len = data[8].to(device)
      
      

      output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
      )
      
      
      
      loss1 = loss_cls(output_cls.view(-1),torch.tensor([mpp_targets,ml_targets],dtype = torch.float).to(device))
      
      loss2 = loss_reg(output_sen1,first_targets.view(-1))
      
      loss3 = loss_reg(ouptut_sen2,second_targets.view(-1))
      
      loss = loss1
 
      
      

      train_loss.append(loss.item())
      
      
      
      if output_cls[0]>0.5:
        train_mpp_predictions.append(1)
      else:
        train_mpp_predictions.append(0)
      if output_cls[1]>0.5:
        train_ml_predictions.append(1)
      else:
        train_ml_predictions.append(0)

      real_train_mpp.append(mpp_targets.cpu().item())
      real_train_ml.append(ml_targets.cpu().item())
    
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model.eval()

    with torch.no_grad():
      for data in validation_loader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_ids = data[2].to(device)
      
        mpp_targets = data[3].to(device)
        ml_targets = data[4].to(device)
        first_targets = data[5].to(device)
        second_targets = data[6].to(device)
        first_sen_len = data[7].to(device)
        second_sen_len = data[8].to(device)
      

        output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
        )
        loss1 = loss_cls(output_cls,torch.tensor([mpp_targets,ml_targets],dtype = torch.float).to(device))
        
        # loss2 = loss_reg(output_sen1,first_targets.view(-1))
        
        # loss3 = loss_reg(ouptut_sen2,second_targets.view(-1))
        
        loss = loss1
 
      
      

        validation_loss.append(loss.item())
      
        if output_cls[0]>0.5:
          validation_mpp_predictions.append(1)
        else:
          validation_mpp_predictions.append(0)
        if output_cls[1]>0.5:
          validation_ml_predictions.append(1)
        else:
          validation_ml_predictions.append(0)
       

        real_validation_mpp.append(mpp_targets.cpu().item())
        real_validation_ml.append(ml_targets.cpu().item())
    
    train_mpp_accuracy = accuracy_score(real_train_mpp, train_mpp_predictions)
    train_ml_accuracy = accuracy_score(real_train_ml, train_ml_predictions)
    validation_mpp_accuracy = accuracy_score(real_validation_mpp, validation_mpp_predictions)
    validation_ml_accuracy = accuracy_score(real_validation_ml, validation_ml_predictions)
    
    final_train_loss = np.mean(train_loss)
 
    final_validation_loss = np.mean(validation_loss)


    print(
'Epoch [{}] Train Totol Loss: {:.4f}, Train MPP Accuracy: {:.4f}, Train ML Accuracy: {:.4f}, Valid Total Loss: {:.4f}, Valid MPP Accuracy: {:.4f}, Valid ML Accuracy: {:.4f}'
    .format(epo+1,final_train_loss, train_mpp_accuracy, train_ml_accuracy,final_validation_loss,validation_mpp_accuracy,validation_ml_accuracy))

In [ ]:
random_seed = [171,354,550,667,985]
for i in random_seed:
  
  df = shuffle(df,random_state=i)

  
  train = df[0:156]

  test = df[156:200]

  train_post1 = train['post1_en']
  train_post2 = train['post2_en']
  train_mpp_label = train['MPP_label']
  train_ml_label = train['ML_label']
  train_mpp_value1 = train['MPP1'] 
  train_mpp_value2 = train['MPP2']
  train_ml_value1 = train['ML1']
  train_ml_value2 = train['ML2']

  test_post1 = test['post1_en']
  test_post2 = test['post2_en']
  test_mpp_label = test['MPP_label']
  test_ml_label = test['ML_label']
  test_mpp_value1 = test['MPP1'] 
  test_mpp_value2 = test['MPP2']
  test_ml_value1 = test['ML1']
  test_ml_value2 = test['ML2']

  max_len = 256
  batch_size = 1
  train_dataloader = generate_dataloader(train_post1,train_post2,train_mpp_label,train_ml_label,max_len,train_mpp_value1,train_ml_value1,train_mpp_value2,train_ml_value2,batch_size)
  test_dataloader = generate_dataloader(test_post1,test_post2,test_mpp_label,test_ml_label,max_len,test_mpp_value1,test_ml_value1,test_mpp_value2,test_ml_value2,batch_size)

  label_num_cls = 2
  label_num_reg = 2
  model= BertNerClassifier(label_num_cls,label_num_reg).to(device)
  optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)
  
  
  train_and_eval(model, 
    train_dataloader, 
    test_dataloader, 
    optimizer,  
    device,
    EPOCHS,
    )




Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 0.7368, Train MPP Accuracy: 0.4615, Train ML Accuracy: 0.5000, Valid Total Loss: 0.7068, Valid MPP Accuracy: 0.5250, Valid ML Accuracy: 0.5000
Epoch [2] Train Totol Loss: 0.5336, Train MPP Accuracy: 0.8141, Train ML Accuracy: 0.8526, Valid Total Loss: 0.7152, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.6500
Epoch [3] Train Totol Loss: 0.3783, Train MPP Accuracy: 0.9808, Train ML Accuracy: 0.9615, Valid Total Loss: 0.7316, Valid MPP Accuracy: 0.5250, Valid ML Accuracy: 0.7000
Epoch [4] Train Totol Loss: 0.2457, Train MPP Accuracy: 1.0000, Train ML Accuracy: 0.9936, Valid Total Loss: 0.7553, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.6750
Epoch [5] Train Totol Loss: 0.1511, Train MPP Accuracy: 1.0000, Train ML Accuracy: 1.0000, Valid Total Loss: 0.7850, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.6750
Epoch [6] Train Totol Loss: 0.0940, Train MPP Accuracy: 1.0000, Train ML Accuracy: 1.0000, Valid Total Loss: 0.8179, Valid MPP Accuracy: 0.5000, 

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 0.7287, Train MPP Accuracy: 0.4936, Train ML Accuracy: 0.4551, Valid Total Loss: 0.7200, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.4750
Epoch [2] Train Totol Loss: 0.5222, Train MPP Accuracy: 0.8397, Train ML Accuracy: 0.8205, Valid Total Loss: 0.7393, Valid MPP Accuracy: 0.4000, Valid ML Accuracy: 0.4500
Epoch [3] Train Totol Loss: 0.3567, Train MPP Accuracy: 0.9872, Train ML Accuracy: 0.9679, Valid Total Loss: 0.7715, Valid MPP Accuracy: 0.4250, Valid ML Accuracy: 0.4250
Epoch [4] Train Totol Loss: 0.2194, Train MPP Accuracy: 1.0000, Train ML Accuracy: 0.9936, Valid Total Loss: 0.8060, Valid MPP Accuracy: 0.4250, Valid ML Accuracy: 0.4000
Epoch [5] Train Totol Loss: 0.1308, Train MPP Accuracy: 1.0000, Train ML Accuracy: 1.0000, Valid Total Loss: 0.8433, Valid MPP Accuracy: 0.4250, Valid ML Accuracy: 0.4250
Epoch [6] Train Totol Loss: 0.0816, Train MPP Accuracy: 1.0000, Train ML Accuracy: 1.0000, Valid Total Loss: 0.8830, Valid MPP Accuracy: 0.4250, 

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 0.7343, Train MPP Accuracy: 0.4936, Train ML Accuracy: 0.5064, Valid Total Loss: 0.7134, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.5500
Epoch [2] Train Totol Loss: 0.5157, Train MPP Accuracy: 0.8205, Train ML Accuracy: 0.8397, Valid Total Loss: 0.7300, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.5500
Epoch [3] Train Totol Loss: 0.3531, Train MPP Accuracy: 0.9615, Train ML Accuracy: 0.9679, Valid Total Loss: 0.7613, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.5500
Epoch [4] Train Totol Loss: 0.2198, Train MPP Accuracy: 1.0000, Train ML Accuracy: 1.0000, Valid Total Loss: 0.8069, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.5250
Epoch [5] Train Totol Loss: 0.1316, Train MPP Accuracy: 1.0000, Train ML Accuracy: 1.0000, Valid Total Loss: 0.8577, Valid MPP Accuracy: 0.5250, Valid ML Accuracy: 0.5250
Epoch [6] Train Totol Loss: 0.0819, Train MPP Accuracy: 1.0000, Train ML Accuracy: 1.0000, Valid Total Loss: 0.9116, Valid MPP Accuracy: 0.5250, 

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 0.7301, Train MPP Accuracy: 0.5321, Train ML Accuracy: 0.4487, Valid Total Loss: 0.6801, Valid MPP Accuracy: 0.5250, Valid ML Accuracy: 0.5250
Epoch [2] Train Totol Loss: 0.5312, Train MPP Accuracy: 0.8077, Train ML Accuracy: 0.8205, Valid Total Loss: 0.6910, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.5500
Epoch [3] Train Totol Loss: 0.3614, Train MPP Accuracy: 0.9359, Train ML Accuracy: 0.9808, Valid Total Loss: 0.7163, Valid MPP Accuracy: 0.4750, Valid ML Accuracy: 0.5500
Epoch [4] Train Totol Loss: 0.2227, Train MPP Accuracy: 0.9936, Train ML Accuracy: 1.0000, Valid Total Loss: 0.7550, Valid MPP Accuracy: 0.4250, Valid ML Accuracy: 0.5250
Epoch [5] Train Totol Loss: 0.1347, Train MPP Accuracy: 1.0000, Train ML Accuracy: 1.0000, Valid Total Loss: 0.8016, Valid MPP Accuracy: 0.4500, Valid ML Accuracy: 0.5250
Epoch [6] Train Totol Loss: 0.0848, Train MPP Accuracy: 1.0000, Train ML Accuracy: 1.0000, Valid Total Loss: 0.8541, Valid MPP Accuracy: 0.4750, 

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Totol Loss: 0.7219, Train MPP Accuracy: 0.5128, Train ML Accuracy: 0.4872, Valid Total Loss: 0.7291, Valid MPP Accuracy: 0.5000, Valid ML Accuracy: 0.5000
Epoch [2] Train Totol Loss: 0.5096, Train MPP Accuracy: 0.7756, Train ML Accuracy: 0.8590, Valid Total Loss: 0.7538, Valid MPP Accuracy: 0.4500, Valid ML Accuracy: 0.4750
Epoch [3] Train Totol Loss: 0.3361, Train MPP Accuracy: 0.9487, Train ML Accuracy: 0.9808, Valid Total Loss: 0.8087, Valid MPP Accuracy: 0.5250, Valid ML Accuracy: 0.4500
Epoch [4] Train Totol Loss: 0.2037, Train MPP Accuracy: 0.9936, Train ML Accuracy: 1.0000, Valid Total Loss: 0.8840, Valid MPP Accuracy: 0.5250, Valid ML Accuracy: 0.4500
Epoch [5] Train Totol Loss: 0.1236, Train MPP Accuracy: 1.0000, Train ML Accuracy: 1.0000, Valid Total Loss: 0.9929, Valid MPP Accuracy: 0.4250, Valid ML Accuracy: 0.5250
Epoch [6] Train Totol Loss: 0.0775, Train MPP Accuracy: 1.0000, Train ML Accuracy: 1.0000, Valid Total Loss: 1.0650, Valid MPP Accuracy: 0.4250, 

In [ ]:
def test_prediction(model, 
  test_dataloader,  
  optimizer,
  device
  ):
  
  test_mpp_predictions = []
  test_ml_predictions = []
  real_test_mpp = []
  real_test_ml= []

  

  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_ids = data[2].to(device)
      
        mpp_targets = data[3].to(device)
        ml_targets = data[4].to(device)
        first_targets = data[5].to(device)
        second_targets = data[6].to(device)
        first_sen_len = data[7].to(device)
        second_sen_len = data[8].to(device)
      

        output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
        )
     
        
      
        if output_cls[0]>0.5:
          test_mpp_predictions.append(1)
        else:
          test_mpp_predictions.append(0)
        if output_cls[1]>0.5:
          test_ml_predictions.append(1)
        else:
          test_ml_predictions.append(0)
        # validation_ml_predictions += torch.sum(preds == ml_targets)

        real_test_mpp.append(mpp_targets.cpu().item())
        real_test_ml.append(ml_targets.cpu().item())

        test_mpp_accuracy = accuracy_score(real_test_mpp, test_mpp_predictions)
        test_ml_accuracy = accuracy_score(real_test_ml, test_ml_predictions)

  

  print('Test MMP ACCURACY_SCORE: {:.4f},Test ML ACCURACY_SCORE: {:.4f}'.format(test_mpp_accuracy,test_ml_accuracy))

In [ ]:
test_prediction(model, 
  test_dataloader, 
  optimizer,
  device,
  
  )

NameError: ignored

In [ ]:
model.load_state_dict(torch.load('fincom_ml_bert_state.bin', map_location=torch.device("cuda:0")))

<All keys matched successfully>

In [ ]:
import googletrans

print(googletrans.LANGUAGES)
from googletrans import Translator
translator = Translator()

# result = Translator.translate('Mikä on nimesi', dest='fr')

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat